In [1]:
!pip install recbole ray > None

In [2]:
import numpy as np
import pandas as pd
import dill

import warnings
warnings.filterwarnings('ignore')

import logging
from logging import getLogger
from recbole.config import Config
from recbole.data import create_dataset, data_preparation
from recbole.model.sequential_recommender import GRU4Rec, Caser
from recbole.trainer import Trainer
from recbole.utils import init_seed, init_logger
from recbole.quick_start import run_recbole

import time

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [5]:
%cd gdrive/MyDrive/recsys/

/content/gdrive/MyDrive/recsys


In [6]:
SEED = 2022
K_RECOS = 10

## Get data

In [7]:
def load_data():
  interactions = pd.read_csv('kion_train/interactions.csv')
  users = pd.read_csv('kion_train/users.csv')
  items = pd.read_csv('kion_train/items.csv')
  interactions.rename(
    columns={
        'last_watch_dt': 'datetime',
        'total_dur': 'weight',
    },
    inplace=True,
  )

  interactions['datetime'] = pd.to_datetime(interactions['datetime'], format="%Y-%m-%d")
  interactions['timestamp'] = interactions.datetime.values.astype(np.int64) // 10 ** 9

  return interactions, users, items

In [8]:
!mkdir recbox_data

mkdir: cannot create directory ‘recbox_data’: File exists


In [9]:
interactions, users, items = load_data()

In [10]:
df = interactions[['user_id', 'item_id', 'timestamp']].rename(
    columns={
        'user_id': 'user_id:token',
        'item_id': 'item_id:token',
        'timestamp': 'timestamp:float'
    }
)
df.to_csv('recbox_data/recbox_data.inter', index=False, sep='\t')

In [ ]:
parameter_dict = {
    'data_path': '',
    'USER_ID_FIELD': 'user_id',
    'ITEM_ID_FIELD': 'item_id',
    'TIME_FIELD': 'timestamp',
    'device': 'GPU',
    'user_inter_num_interval': "[40,inf)",
    'item_inter_num_interval': "[40,inf)",
    'load_col': {'inter': ['user_id', 'item_id', 'timestamp']},
    'neg_sampling': None,
    'epochs': 30,
    'eval_args': {
        'split': {'RS': [8, 1, 1]},
        'group_by': 'user',
        'order': 'TO',
        'mode': 'full',
    },
    'metrics': ['MAP', 'NDCG', 'Recall'],
    'valid_metric': 'MAP@10',
    'topk': 10,
    'seed': 2022,
    'eval_step': 12,
}
config = Config(model='MultiVAE', dataset='recbox_data', config_dict=parameter_dict)

init_seed(config['seed'], config['reproducibility'])


init_logger(config)
logger = getLogger()
c_handler = logging.StreamHandler()
c_handler.setLevel(logging.INFO)
logger.addHandler(c_handler)

logger.info(config)

In [ ]:
dataset = create_dataset(config)
logger.info(dataset)

In [ ]:
train_data, valid_data, test_data = data_preparation(config, dataset)

## Trying to find best model

In [ ]:
%%time
model_list = ['MultiVAE', 'CDAE', 'ENMF', 'RecVAE', 'NNCF', 'RaCT'] 

for model_name in model_list:
    print(f"running {model_name}...")
    start = time.time()
    result = run_recbole(model=model_name, dataset='recbox_data', config_dict=parameter_dict)
    t = time.time() - start
    print(f"It took {t / 60:.2f} mins")
    print(result)

running MultiVAE...


Evaluate   : 100%|███████████████████| 13354/13354 [04:19<00:00, 51.56it/s, GPU RAM: 0.34 G/14.76 G]


It took 14.09 mins
{'best_valid_score': 0.0364, 'valid_score_bigger': True, 'best_valid_result': OrderedDict([('map@10', 0.0364), ('ndcg@10', 0.0776), ('recall@10', 0.0878)]), 'test_result': OrderedDict([('map@10', 0.0337), ('ndcg@10', 0.0703), ('recall@10', 0.0765)])}
running CDAE...


Evaluate   : 100%|███████████████████| 13354/13354 [05:46<00:00, 38.51it/s, GPU RAM: 0.37 G/14.76 G]


It took 18.60 mins
{'best_valid_score': 0.0329, 'valid_score_bigger': True, 'best_valid_result': OrderedDict([('map@10', 0.0329), ('ndcg@10', 0.0645), ('recall@10', 0.0633)]), 'test_result': OrderedDict([('map@10', 0.0335), ('ndcg@10', 0.0653), ('recall@10', 0.0625)])}
running ENMF...


...

Evaluate   : 100%|███████████████████| 13354/13354 [09:45<00:00, 22.82it/s, GPU RAM: 1.59 G/14.76 G]


It took 64.75 mins
{'best_valid_score': 0.037, 'valid_score_bigger': True, 'best_valid_result': OrderedDict([('map@10', 0.037), ('ndcg@10', 0.0767), ('recall@10', 0.0814)]), 'test_result': OrderedDict([('map@10', 0.0363), ('ndcg@10', 0.0741), ('recall@10', 0.0761)])}
running RaCT...


Evaluate   : 100%|███████████████████| 13354/13354 [09:02<00:00, 24.60it/s, GPU RAM: 1.59 G/14.76 G]


It took 11.43 mins
{'best_valid_score': -inf, 'valid_score_bigger': True, 'best_valid_result': None, 'test_result': OrderedDict([('map@10', 0.029), ('ndcg@10', 0.0623), ('recall@10', 0.0687)])}
running DGCF...


Train     7: 100%|███████████████████████| 339/339 [06:29<00:00,  1.15s/it, GPU RAM: 4.40 G/14.76 G]


The best model is **RecVAE**

## Refit best model

In [14]:
parameter_dict = {
    'data_path': '',
    'USER_ID_FIELD': 'user_id',
    'ITEM_ID_FIELD': 'item_id',
    'TIME_FIELD': 'timestamp',
    'device': 'GPU',
    'user_inter_num_interval': "[40,inf)",
    'item_inter_num_interval': "[40,inf)",
    'load_col': {'inter': ['user_id', 'item_id', 'timestamp']},
    'neg_sampling': None,
    'epochs': 60,
    'eval_args': {
        'split': {'RS': [8, 1, 1]},
        'group_by': 'user',
        'order': 'TO',
        'mode': 'full',
    },
    'metrics': ['MAP', 'NDCG', 'Recall'],
    'valid_metric': 'MAP@10',
    'topk': 10,
    'seed': 2022,
    'eval_step': 20,
}
config = Config(model='RecVAE', dataset='recbox_data', config_dict=parameter_dict)

init_seed(config['seed'], config['reproducibility'])


init_logger(config)
logger = getLogger()
c_handler = logging.StreamHandler()
c_handler.setLevel(logging.INFO)
logger.addHandler(c_handler)

logger.info(config)

In [15]:
dataset = create_dataset(config)
logger.info(dataset)

In [16]:
train_data, valid_data, test_data = data_preparation(config, dataset)

In [17]:
result = run_recbole(model='RecVAE', dataset='recbox_data', config_dict=parameter_dict )

Evaluate   : 100%|███████████████████████████████████████████| 13354/13354 [00:45<00:00, 290.61it/s]


In [18]:
result

{'best_valid_score': 0.0395,
 'valid_score_bigger': True,
 'best_valid_result': OrderedDict([('map@10', 0.0395),
              ('ndcg@10', 0.0833),
              ('recall@10', 0.0921)]),
 'test_result': OrderedDict([('map@10', 0.0353),
              ('ndcg@10', 0.0738),
              ('recall@10', 0.0792)])}